# Report Section

**Goal:**
In this project, I aim to predict the domain such as Archaea, Bacteria, Eukaryota or Virus from the abstract of research papers about proteins taken from the MELINE database by implementing null model, standard Naive Bayes Classifier and its extended algorithm (Log_probability and TF-IDF). 
   
**Datasets:**
There are 4000 abstracts with imbalanced classes in trg.csv file; there are 1000 abstracts without classes in tst.csv file.

**Representation and Data pre-processing:**
First of all, I eliminate the punctuations (i.e. dash and prime), stop words (hard-coding) and words which its length is less than 3 from the original 4000 abstracts, I also singularizes the words. As they are all lowercase characters, I don’t need to convert them to lowercase. For the late model evaluation/validation, because the dataset is imbalanced, I implement stratified train/test split (80:20) based on the propostion of each class before the abstracts’ text processing to avoid data leakage (golden rule of Machine Learning). I randomly choose 3200 abstracts as a training set and extract unique words from those 3200 abstracts as my vocabulary for each run of train/test split (I experiment 100 runs, so I have different vocabulary each run). The vocabulary is considered the features/columns. I mapped the abstracts from both the 3200 training set and 800 test set to the vocabulary with frequencies each word of vocabulary occurs in each abstract. So the row is represented as abstracts, and the column is represented as each word in the vocabulary; the value is represented as the frequencies. We will have both training and test set in this setting of representation.

**Implementation of null model:**
To find the most frequency class in training dataset, and labels all the test data as the most frequency class.

**Implementation of standard Naive Bayes:**
As Naïve Bayes classifier assumes, which the event each word occurs is conditionally independent by each other given the class. We use MAP (maximum a posterior) to predict the class.  First, I calculate the prior probabilities for each class. Secondly, I calculate the conditional probabilities, which is the likelihood of each word given each class. I also apply Laplace smoothing to this step to avoid the problem of zero probability. For each abstract, I use the pre-calculated prior probabilities and conditional probabilities to multiply them together for each class (‘A’, ‘B’, ‘E’, ‘V’), and predict the class with the highest product.  

**Implementation of Naive Bayes extended with Log_probabillty and TF-IDF:**
As the poor performance of the standard Naïve Bayes implementation and its performance is significantly different from the null model, which takes the majority class as predicition, it suffers from underflow issue extremaly, literally, the prodct of probabities is too small, which returns 0 value, so I use log probabilities to fix this issue. Basically, I take a log of each probability - prior probabilities and condiaitonal probabilities. By the handy property of logarithm, I can find the MAP by summation of the log probabilities instead of product of the probabilies. 
The standard Naïve Bayes also has a problem if a word appears in every abstract which is obviously meaningless for our classification. So I introduce the TF-IDF to my extended algorithm. Bacially I pre-calcualte IDF (inverse document frequency) term initially and take a log of them, I replace the original frequencies term by the product of original requencies and IDF term as new frequencies.  The rest of steps is the same as the standard Naïve Bayes.

**Results:**
I run stratified cross-validation 100 times and use paired sample t-test to test whether there is a significant difference among the performance of the null model, standard Naive Bayes and the extended version. 
1. As the result from the **cell [14]**, it is showing there is a significant difference (t_score = -273.191 and df = 99) between extended Naïve Bayes and standard Naïve Bayes , and the **cell [12]** is showing the extended Naïve Bayes (average accuracy = 0.9688) outperforms than the standard Naïve Bayes (average accuracy = 0.5187). 
2. As the result from the **cell [15]**, it is showing there is a significant difference (t_score = -818.752 and df = 99) between extended Naïve Bayes and null model, and the **cell [12]** is showing the extended Naïve Bayes (average accuracy = 0.9688) outperforms than the null model (average accuracy = 0.5363). 
3. As the result from the **cell [16]**, it is showing there is a significant difference (t_score = 11.276 and df = 99) between standard Naïve Bayes and null model, and the **cell [12]** is showing the null model (average accuracy = 0.5363) outperforms than the standard Naïve Bayes (average accuracy = 0.5187).

**Kaggle submission:**
My highest score is 0.97666    

# Code Section

## Loading essential packages
- numpy

In [1]:
import numpy as np

## Loading datasets

In [2]:
punctuations = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\n'

## Loading trg.csv by columns 
id_trg = []
class_trg = []
abstracts_trg = []
with open(r'C:\Users\jovi1\OneDrive - The University of Auckland\Documents\COMPSCI 762_2021\Assignment 3\trg.csv', 'r') as f1:
    lines = f1.readlines()[1:]
    for line in lines:
        row = line.split(',')
        id_trg.append(row[0])
        class_trg.append(row[1])
        abstracts_trg.append(''.join([c for c in row[2] if c not in punctuations]))
id_trg = np.array(id_trg)
class_trg = np.array(class_trg)
abstracts_trg = np.array(abstracts_trg)

## Loading tst.csv by columns       
id_tst = []
abstracts_tst = []
with open(r'C:\Users\jovi1\OneDrive - The University of Auckland\Documents\COMPSCI 762_2021\Assignment 3\tst.csv', 'r') as f2:
    lines = f2.readlines()[1:]
    for line in lines:
        row = line.split(',')
        id_tst.append(row[0])
        abstracts_tst.append(''.join([c for c in row[1] if c not in punctuations]))
id_tst = np.array(id_tst)
abstracts_tst = np.array(abstracts_tst)

# Task 1

## implement text processing
- Create a dictionary for all the distinct words with their occurrence frequencies.
- Remove the stop words from the text in all abstracts.
- Remove the digits
- Remove the words whose length is less than three
- Singularize the words

In [3]:
def singularizer(word):
    if len(word) > 3: 
        if word[-2:] in 'es' and (word[-4:-2] in ['sh', 'ch'] or word[-3] in ['s', 'x', 'z']):
            word = word[:-2]
        elif word[-1] == 's' and (word[-4:-2] not in ['sh', 'ch'] or word[-3] not in ['s', 'x', 'z']):
            word = word[:-1]
        elif word[-3:] == 'ies':
            word = word.replace('ies', 'y')
        elif word[-3:] == 'ves':
            word = word.replace('ves', 'f')
            if word[-2] == 'i':
                word = word.replace('f', 'fe')
            else:
                pass
        elif word[-3:] == 'men':
            word = word.replace('men', 'man')
        else:
            pass
        return word
    else:
        return word

In [4]:
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", 
              "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 
              'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 
              'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 
              'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 
              'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 
              'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 
              'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 
              'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 
              'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 
              'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 
              'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 
              'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should',  
              "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 
              'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', 
              "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', 
              "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', 
              "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]


def get_vocabulary(abstracts):
    unique_words = set()
    for abstract in abstracts:
        for word in abstract.split():
            word = singularizer(word)
            if word not in stop_words and not word.isdigit() and len(word) > 2:
                unique_words.add(word)

    vocab = {}
    for index, word in enumerate(sorted(list(unique_words))):
        vocab[word] = index
    return vocab

In [5]:
def Count_Vectorizer(abstracts, vocabulary):
    row, col, val = [], [], []
    for row_index, abstract in enumerate(abstracts):
        count_word = {}
        for word in abstract.split():
            word = singularizer(word)
            if word in count_word:
                count_word[word] += 1
            else:
                count_word[word] = 1
        for word, count in count_word.items():
            col_index = vocabulary.get(word)
            if col_index != None:
                row.append(row_index)
                col.append(col_index)
                val.append(count)
    mat = np.zeros([len(abstracts), len(vocabulary)])
    mat[row, col] = val
    return mat

## implement stratified train/test set split function 

In [6]:
# stratified train_test_split 
def train_test_split(X, y, test_size, random_state = 1):
    np.random.seed(random_state)
    unique, counts = np.unique(y, return_counts=True)
    test_index = np.array([], dtype='int')
    train_index = np.array([], dtype='int')
    for i in range(len(unique)): 
        test_num = int(counts[i] * test_size)
        ind = np.where(y == unique[i])[0]
        test_i = np.random.choice(ind, test_num, replace=False)
        train_i = np.setxor1d(ind, test_i)
        test_index = np.append(test_index, test_i)
        train_index = np.append(train_index, train_i)
    return X[train_index], X[test_index], y[train_index], y[test_index]

## implement performance metrics
- over-all accuracy

In [7]:
def performance_metric(y_test, y_pred):
    return (y_test == y_pred).sum() / len(y_test)

# Task 2

## implement null model

In [8]:
# null model takes the majority class as class prediction.
def null_model(y_train, X_test):
    unique, counts = np.unique(y_train, return_counts=True)
    y_pred = [unique[np.argmax(counts)]] * len(X_test) 
    return np.array(y_pred)

## implement Standard/Multinomial Navie Bayes classifier 

In [9]:
def Naive_Bayes(X_train, y_train, X_test):
    voc_size = X_train.shape[1] 
    
    # calculate the priors for each class
    prior_A = np.sum(y_train=='A')/len(y_train)
    prior_B = np.sum(y_train=='B')/len(y_train)
    prior_E = np.sum(y_train=='E')/len(y_train)
    prior_V = np.sum(y_train=='V')/len(y_train)
    
    n_words_A = np.sum(X_train[y_train=='A'])
    n_words_B = np.sum(X_train[y_train=='B'])
    n_words_E = np.sum(X_train[y_train=='E'])
    n_words_V = np.sum(X_train[y_train=='V'])
    
    # calculate the conditional probability
    # Add one to the numerator and add the number of vocabulary to elminate zeros 
    con_prob_A = (np.sum(X_train[y_train=='A'], axis=0) + 1) / (n_words_A + voc_size)
    con_prob_B = (np.sum(X_train[y_train=='B'], axis=0) + 1) / (n_words_B + voc_size)
    con_prob_E = (np.sum(X_train[y_train=='E'], axis=0) + 1) / (n_words_E + voc_size)
    con_prob_V = (np.sum(X_train[y_train=='V'], axis=0) + 1) / (n_words_V + voc_size)
    
    y_pred = []
    for i in range(len(X_test)):
        ind = np.nonzero(X_test[i])[0]

        score_A = prior_A * np.prod(np.power(con_prob_A[ind], X_test[i][ind]))
        score_B = prior_B * np.prod(np.power(con_prob_B[ind], X_test[i][ind]))
        score_E = prior_E * np.prod(np.power(con_prob_E[ind], X_test[i][ind]))
        score_V = prior_V * np.prod(np.power(con_prob_V[ind], X_test[i][ind]))

        if max([score_A, score_B, score_E, score_V]) == score_A:
            y_pred.append('A')
        elif max([score_A, score_B, score_E, score_V]) == score_B:
            y_pred.append('B')
        elif max([score_A, score_B, score_E, score_V]) == score_E:
            y_pred.append('E')
        else:
            y_pred.append('V')

    return np.array(y_pred)   

## Naive Bayes classifier extended with log probability and TF-IDF

In [10]:
def Naive_Bayes_ext(X_train, y_train, X_test):
    voc_size = X_train.shape[1] 
    
    # Calculate IDF term
    IDF = np.log(len(X_train) / np.sum(X_train!=0, axis=0))  
    
    # Calculate the priors for each class
    prior_A = np.sum(y_train=='A')/len(y_train)
    prior_B = np.sum(y_train=='B')/len(y_train)
    prior_E = np.sum(y_train=='E')/len(y_train)
    prior_V = np.sum(y_train=='V')/len(y_train)
    
    n_words_A = np.sum(X_train[y_train=='A'] * IDF)
    n_words_B = np.sum(X_train[y_train=='B'] * IDF)
    n_words_E = np.sum(X_train[y_train=='E'] * IDF)
    n_words_V = np.sum(X_train[y_train=='V'] * IDF)
    
    # calculate the conditional probability
    # Add one to the numerator and add the number of vocabulary to elminate zeros 
    con_prob_A = (np.sum(X_train[y_train=='A'], axis=0) * IDF + 1) / (n_words_A + voc_size)
    con_prob_B = (np.sum(X_train[y_train=='B'], axis=0) * IDF + 1) / (n_words_B + voc_size)
    con_prob_E = (np.sum(X_train[y_train=='E'], axis=0) * IDF + 1) / (n_words_E + voc_size)
    con_prob_V = (np.sum(X_train[y_train=='V'], axis=0) * IDF + 1) / (n_words_V + voc_size)
    
    y_pred = []
    for i in range(len(X_test)):
        ind = np.nonzero(X_test[i])[0]
        score_A = np.log10(prior_A) + np.sum(np.multiply(np.log10(con_prob_A[ind]), X_test[i][ind]))
        score_B = np.log10(prior_B) + np.sum(np.multiply(np.log10(con_prob_B[ind]), X_test[i][ind]))
        score_E = np.log10(prior_E) + np.sum(np.multiply(np.log10(con_prob_E[ind]), X_test[i][ind]))
        score_V = np.log10(prior_V) + np.sum(np.multiply(np.log10(con_prob_V[ind]), X_test[i][ind]))
        if max([score_A, score_B, score_E, score_V]) == score_A:
            y_pred.append('A')
        elif max([score_A, score_B, score_E, score_V]) == score_B:
            y_pred.append('B')
        elif max([score_A, score_B, score_E, score_V]) == score_E:
            y_pred.append('E')
        else:
            y_pred.append('V')

    return np.array(y_pred)   

## Cross Validation
- 100 times

In [11]:
null = []
nb = []
nb_ext = []

for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(abstracts_trg, class_trg, test_size=0.2, random_state=i)
    
    vocabulary = get_vocabulary(X_train)
    
    X_train_cv = Count_Vectorizer(X_train, vocabulary)
    X_test_cv = Count_Vectorizer(X_test, vocabulary)
    
    y_null = null_model(y_train, X_test)
    null.append(performance_metric(y_test, y_null))
    
    y_nb = Naive_Bayes(X_train_cv, y_train, X_test_cv)
    nb.append(performance_metric(y_test, y_nb))

    y_nb_ext = Naive_Bayes_ext(X_train_cv, y_train, X_test_cv)
    nb_ext.append(performance_metric(y_test, y_nb_ext))

    
null = np.array(null)  
nb = np.array(nb)
nb_ext = np.array(nb_ext)

In [12]:
print('The average of accuracies for null model is {}, and the standard devation is {}'.format(round(np.mean(null), 4), round(np.std(null), 4)))
print('The average of accuracies for standard naive bayes is {}, and the standard devation is {}'.format(round(np.mean(nb), 4), round(np.std(nb), 4)))
print('The average of accuracies for extended version of naive bayes is {}, and the standard devation is {}'.format(round(np.mean(nb_ext), 4), round(np.std(nb_ext), 4)))

The average of accuracies for null model is 0.5363, and the standard devation is 0.0
The average of accuracies for standard naive bayes is 0.5187, and the standard devation is 0.0155
The average of accuracies for extended version of naive bayes is 0.9688, and the standard devation is 0.0053


## paired sample t-test

In [13]:
def paired_sample_t_score(sample_1, sample_2, size = 100):
    mean_1 = np.mean(sample_1)
    mean_2 = np.mean(sample_2)
    var_1 = np.sum((sample_1 - mean_1) ** 2) / (size - 1)
    var_2 = np.sum((sample_2 - mean_2) ** 2) / (size - 1)
    std = np.sqrt((var_1 + var_2) / 2)
    t_score = (mean_1 - mean_2) / (std * np.sqrt(2 / size))
    return t_score

### comparison between extended version and standard Naive Bayes

In [14]:
t_score = paired_sample_t_score(nb, nb_ext)
print('t score: {}'.format(round(t_score, 3)))

t score: -273.191


At 0.95 significant level, the p-value is < 0.00001 with degree of freedom = 99. So we can conclude the Navie Bayes with log probability and TF-IDF outperforms the standard Navie Bayes.

### comparison between extended version and null model

In [15]:
t_score = paired_sample_t_score(null, nb_ext)
print('t score: {}'.format(round(t_score, 3)))

t score: -818.752


At 0.95 significant level, the p-value is < 0.00001 with degree of freedom = 99. So we can conclude the Navie Bayes with log probability and TF-IDF outperforms the null model.

### comparison between standard Naive Bayes and null model

In [16]:
t_score = paired_sample_t_score(null, nb)
print('t score: {}'.format(round(t_score, 3)))

t score: 11.276


At 0.95 significant level, the p-value is < 0.00001 with degree of freedom = 99. So we can conclude the null model outperforms the standard Naive Bayes.

## Kaggle submission

In [17]:
vocabulary = get_vocabulary(abstracts_trg)
X_tst = Count_Vectorizer(abstracts_tst, vocabulary=vocabulary)

In [18]:
X = Count_Vectorizer(abstracts_trg, vocabulary=vocabulary)
y = class_trg
y_pred = Naive_Bayes_ext(X, y, X_tst)

## save the output as csv file, and upload it for Kaggle Competition 

In [19]:
output = np.array([id_tst, y_pred]).T
output = np.vstack((['id', 'class'], output))

with open('output.csv', 'w') as f3:
    f3.write("\n".join(",".join(map(str, x)) for x in output))